# Contents
1. [Defining Types](#define)
1. [Algebraic Types](#algebraic)
1. [Pattern Matching](#pattern)
1. [Record Syntax](#record_syntax)
1. [Parametrized Types](#parametrized_types)
1. [Recursive Types](#recursive_types)
1. [Reporting Errors](#reporting_errors)
1. [Local Variables](#local_vars)
1. [Shadowing](#shadowing)

#### Defining a Type <a id='define'></a>

`BookInfo` is the type constructor, the name of our type. A type name and therefore the constructor must start with a capital letter in Haskell. The `Book` that follows the equals sign is the value constructor, or data constructor. You use this to create a value of `BookInfo` type. This must also start with a capital letter. The types following `Book` are the components of a type, like the fields or attributes in a struct/class in OO languages. It's a *slot* where a value can be inserted.

In [ ]:
-- define a new data type using the data keyword
data BookInfo = Book Int String [String]
                deriving (Show)
                
data MagazineInfo = Magazine Int String [String]
                    deriving (Show)

Even though `MagazineInfo` has the same structure as `BookInfo` Haskell treats these distinct types as separate, because their constructors have different names.

In [ ]:
book1 = Book 123123 "leaves of grass" ["Walt Whitman"]
mag1 = Magazine 123123 "the economist" ["London, England"]

In [ ]:
:type book1

In [ ]:
:type mag1

We can treat value constructors like any other function, it's just one that happens to create and return a value of some type that we want.

In [ ]:
-- type synonyms are like C typedefs
type CustomerID = Int
type ReviewBody = String

-- Type and Values constructors can/normally do have the same name 
data BookReview = BookReview BookInfo CustomerID ReviewBody

#### Algebraic Datatypes <a id='algebraic'></a>

Can have more than one values constructor.

In [ ]:
-- Has two value constructors
data Bool = True | False

In [ ]:
type CardHolder = String
type CardNumber = String
type Address = [String]

data BillingInfo = CreditCard CardNumber CardHolder Address 
                 | CashOnDelivery
                 | Invoice CustomerID
                   deriving (Show)

Saying that there are 3 ways of charging a customer, Credit cards require 3 bits of info, cash on delivery no extra bits of info, and Invoicse require a Customer ID.

Algebraic types allow us to distinbuish between two structurally identical types.

In [ ]:
type Name = String
type Manufacturer = String

data Chairs = Chairs Name Manufacturer
              deriving(Show)
              
data Tables = Tables Name Manufacturer
              deriving(Show)

In [ ]:
c = Chairs "barcalounger" "friends"
t = Tables "dining" "friends"

In [ ]:
:type c

In [ ]:
:type t

#### Pattern Matching <a id='pattern'></a>

- If a type has more than one value constructor, we'd like to know which value constructor was used to create a given value.
- If a value constructor has data components, we'd like to  access these values.

A pattern lets us look inside a value, and bind variables to data it contains.

In [ ]:
-- Haskell lets us define a function a series of equations
-- The patterns follow the function name, and precede the equal sign
myNot True = False
myNot False = True

When we pattern match, we reverse the construction process.

In [ ]:
-- Pattern matching a tuple
third (a, b, c) = c

In [ ]:
third ('a', 2, "three")

There's no limit into how deep this matching can go

In [ ]:
third(1, 1, (1,23))

Can pattern match on an algebraic data type using its value constructors. 

In [ ]:
bookID (Book id title authors) = id
bookTitle(Book id title authors) = title
bookAuthors(Book id title authors) = authors

In [ ]:
bookID (Book 1 "book title" ["a n author"])

In [ ]:
bookAuthors (Book 1 "book title" ["a n author"])

In [ ]:
bookTitle (Book 1 "book title" ["a n author"])

Can use wildcards for pattern matching, more legible.

In [ ]:
betterID (Book id _ _) = id

In [ ]:
betterID(Book 1 "book title" ["a n author"])

#### Record Syntax <a id='record_syntax'></a>

Can define a data type and it's accessors in one go, getting rid of all of the previous boilerplate.

In [ ]:
data Customer = Customer {
      customerID        :: CustomerID
    , customerName      :: String
    , customerAddress   :: Address
    } deriving(Show)

This is exactly the same as below

In [ ]:
data Customer = Customer Int String [String]
                deriving(Show)

customerID :: Customer -> Int
customerID(Customer id _ _) = id

customerName :: Customer -> String
customerName(Customer _ name __ ) = name

customerAddress :: Customer -> [String]
customerAddress(Customer _ _ address) = address

In [ ]:
customer1 = Customer {
        customerID = 123
      , customerAddress = ["123 Fake St"]
      , customerName = "Srinath Kailasa"      
      }

In [ ]:
customer1

In [ ]:
-- compare with book info type from earlier
book1

the accessor functions we get for free are normal Haskell functions

In [ ]:
:type customerName

In [ ]:
customerName customer1

#### Parametrized Types <a id='parametrized_types'></a>

Can also make our types polymorphic, like built-in `List`

In [15]:
data Maybe a = Just a
             | Nothing
               deriving(Show)

`Maybe` is a type that allows us to represent a value that could be present or missing. Above, `a` is a type variable, it indicates that the `Maybe` type takes another type as its paraeter. This lets us use Maybe on values of any type.

In [16]:
:type Just "invisible man"

Just "invisible man" :: Maybe [Char]

In [17]:
Just 1.5

Just 1.5

In [18]:
Just "abc"

Just "abc"

In [23]:
foo:: Maybe String -> String
foo (Just str) = str
foo Nothing = "No string"

In [27]:
a = Just "random string"

In [29]:
b = Just 12

In [28]:
foo a

"random string"

In [30]:
foo b

No instance for (Num String) arising from a use of `b'
In the first argument of `foo', namely `b'
In the expression: foo b
In an equation for `it': it = foo b

`Maybe` is a polymorphic type, we give the type a constructor to create a specific type

If a function doesn't have a meaningful definition for all outputs we can use `Maybe`to say that. Consider the following function to calculate the square roots of real numbers.

In [2]:
import qualified Prelude
import Prelude hiding(sqrt)

sqrt :: Float -> Maybe Float
sqrt x
    | x < 0     = Nothing
    | otherwise = Just (Prelude.sqrt(x))

In [9]:
sqrt 10.0

Just 3.1622777

In [6]:
sqrt (-0.1)

Nothing

#### Recursive Types <a id='recursive_types'></a>

Can define types in terms of themselves

In [ ]:
-- A simple singly linked list
data List a = Cons a (List a)
            | Nil
              deriving(Show)

In [ ]:
Cons 2 (Cons 1 (Cons 0 Nil))

In [11]:
-- A binary tree
data Tree a = Node a (Tree a) (Tree a)
            | Empty
              deriving(Show)

In [12]:
simpleTree = Node "parent" (Node "left child" Empty Empty)
                           (Node "right child" Empty Empty)

`Maybe` sort of acts like generic constructor from other object oriented langauges like C++, here we've replaced the two constructors int the simple Tree with `Maybe`

In [22]:
-- binary tree with a single value constructor
data Tree2 a = Node a (Maybe (Tree2 a)) (Maybe (Tree2 a))
             deriving(Show)

In [23]:
Node 1 (Just (Node 1 Nothing Nothing)) (Just (Node 2 Nothing Nothing))

Node 1 (Just (Node 1 Nothing Nothing)) (Just (Node 2 Nothing Nothing))

#### Reporting Errors <a id='reporting_errors'></a>

Can use the `Maybe` type to represent the possibility of an error. If we want to respresent a failure can use the `Nothing` constructor, otherwise we wrap the values with the `Just` constructor.

In [2]:
second :: [a] -> Maybe a

second [] = Nothing
second xs = if null (tail xs)
            then Nothing
            else Just (head (tail xs))

In [3]:
second [1, 2, 3]

Just 2

In [4]:
second []

Nothing

In [6]:
-- Can improve the readability of this function with pattern matching
tidySecond :: [a] -> Maybe a

tidySecond (_:x:_) = Just x
tidySecond _       = Nothing

The first pattern matches only if the list is at least 2 elements long, and binds `x` to the second element. The second pattern is matched if the first one fails. 

#### Local Variables <a id='local_vars'></a>

Can introduce local variables using `let` expression wherever we need them.

In [8]:
-- Function to decide whether to lend money to a customer
-- with a fundamental reserve amount.
lend amount balance = let reserve    = 100
                          newBalance = balance - amount
                      in if balance < reserve
                         then Nothing
                         else Just newBalance

A name in a `let` block is bound to an expression not a value. This won't be evaluated until it's needed due to laziness. For example `newBalance` would not be calculated if we did not meet our reserve. Variables defined in the `let` block are known as *let*-bound variables.

#### Shadowing <a id='shadowing'></a>

Can nest multiple let blocks within each other.

In [10]:
foo = let a = 1
      in let b = 2
         in a + b

In [11]:
foo

3

It's perfectly legal to redefine variable names in different scopes of `let`, this is known as shadowing. Can also shadow function parameters. 

In [12]:
quux a = let a = "foo"
         in a ++ "eek"

In [13]:
quux "string"

"fooeek"

In [14]:
quux 123

"fooeek"

Means that args can be of any type as `a` is shadowed by let-bound variables.